In [7]:
!pip install 'transformers[sentencepiece]'

In [6]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.40.0
Uninstalling transformers-4.40.0:
  Successfully uninstalled transformers-4.40.0
Found existing installation: accelerate 0.34.2
Uninstalling accelerate-0.34.2:
  Successfully uninstalled accelerate-0.34.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 9.1 MB/s eta 0:00:0000:0100:01m
  Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2


In [10]:
pip install --upgrade datasets


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.6 MB/s eta 0:00:00
  Using cached pycryptodomex-3.20.0-cp35-abi3-macosx_10_9_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 4.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install evaluate rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3ddae6172e69eda26e3de6341826be441348b7c48f9737b788e22fe77d299692
  Stored in directory: /Users/bubnak/Library/Caches/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
from datasets import load_dataset
import evaluate
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
nltk.download("punkt")

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Load the pre-trained model

In [10]:
model = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load Dataset

In [12]:
dataset_samsum = load_dataset("samsum")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [13]:
df = pd.DataFrame(dataset_samsum['train'])

pd.DataFrame(dataset_samsum['train'])

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [14]:
df.dialogue[0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [15]:
df.summary[0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [17]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/Users/bubnak/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [32]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

### Evaluate pre-trained model

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = [[(label if label != -100 else tokenizer.pad_token_id) for label in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    result = {key: value * 100 for key, value in result.items()}  # Return as percentage
    return result


In [40]:
trainer = Seq2SeqTrainer(
    model=model_pegasus,
    args=training_args,
    eval_dataset=dataset_samsum_pt['test'],  # Use the test or validation split
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

eval_results = trainer.evaluate()

print(eval_results)

{'eval_loss': 2.6421968936920166, 'eval_model_preparation_time': 0.0068, 'eval_rouge1': 29.611979328327287, 'eval_rouge2': 8.661650459308024, 'eval_rougeL': 22.759076833540576, 'eval_rougeLsum': 22.785841247859253, 'eval_runtime': 11831.5275, 'eval_samples_per_second': 0.069, 'eval_steps_per_second': 0.069}


### Fine tune pre trained model

In [38]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
trainer_args = TrainingArguments(
        output_dir='pegasus-samsum', 
        num_train_epochs=1, 
        warmup_steps=500,
        per_device_train_batch_size=1, 
        per_device_eval_batch_size=1,
        weight_decay=0.01, 
        logging_steps=10,
        evaluation_strategy='steps', 
        eval_steps=500,  
        save_steps=1e6,
        gradient_accumulation_steps=16
  )

trainer = Trainer(
            model=model_pegasus, 
            args=trainer_args,
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"], 
            eval_dataset=dataset_samsum_pt["validation"])
  
trainer.train()

model_pegasus.save_pretrained("./pegasus-samsum-model")
tokenizer.save_pretrained("./tokenizer")

Step,Training Loss,Validation Loss
500,1.665800,1.484593


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/spiece.model',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

### Evaluate fine tuned model

In [31]:

model = AutoModelForSeq2SeqLM.from_pretrained("./pegasus-samsum-model")
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  
    per_device_eval_batch_size=1, 
    predict_with_generate=True,  
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_samsum_pt['test'],  # Pass your validation set dataset_samsum_pt["validation"]
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

finetuned_results = trainer.evaluate()

print(finetuned_results)


{'eval_loss': 1.4537464380264282, 'eval_model_preparation_time': 0.0067, 'eval_rouge1': 45.04982150447816, 'eval_rouge2': 21.415838982179483, 'eval_rougeL': 35.38904089651658, 'eval_rougeLsum': 35.385210562522005, 'eval_runtime': 7085.8272, 'eval_samples_per_second': 0.116, 'eval_steps_per_second': 0.116}


### Compare difference before and after the fine tuning

In [49]:
for key in metrics_before:
    print(f"Change in {key}: {finetuned_results[key] - eval_results[key]:.2f}")

Change in eval_loss: -1.19
Change in eval_rouge1: 15.44
Change in eval_rouge2: 12.75
Change in eval_rougeL: 12.63


### Generate output from fined tuned model 

In [12]:

model = AutoModelForSeq2SeqLM.from_pretrained("./pegasus-samsum-model")
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

input_text = "Yes, that's correct! When you run a Docker container and bind it to 0.0.0.0:8080, it means: Docker is listening on all available network interfaces: The container can accept connections from any IP address on the host machine.Docker port is set to 8080: This is the port on the host machine that is mapped to the port your application is running on inside the container (for example, if your Flask app is running on port 5000 inside the container, you would typically run the container with -p 8080:5000.So, any requests sent to  will be routed to the application running in the Docker container.The your-host-ip would typically refer to the IP address of the machine where Docker is running. If you're running everything locally (like on your development machine), you can use localhost or 127.0.0.1."
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=128, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated Summary:", summary)

Generated Summary: Docker is listening on all available network interfaces. The container can accept connections from any IP address on the host machine.<n>Docker port is set to 8080.
